In [28]:
import random
import numpy
import copy
from functools import reduce

round_count = 1

height = 20
width = 20
symb_alive = "#"
symb_dead = "."


def create_canvas(height, width):
    game = []
    
    for line in range(height):
        filled_line = []
        for position in range(width):
            filled_line.append(random.randint(0, 1))
        game.append(filled_line)
    
    return game


start_pos = create_canvas(height, width)
print("STARTING POSITION:")
print('\n'.join(map(str, start_pos)))

STARTING POSITION:
[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0]
[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
[0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0]
[0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1]
[0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
[1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0]
[1, 0

In [29]:
game2 = []

def replace(a_list, symb_a, symb_d):
    formatted_line = []
    [formatted_line.append(symb_a) if x == 1 else formatted_line.append(symb_d) for x in a_list]
    return formatted_line

def format_round(game):
    '''creates visualization for a round of game of life. needs round as input'''
    
    game2 = []
    
    for line in game:
        newline = replace(line, symb_alive, symb_dead)
        game2.append("  ".join(str(x) for x in newline))
        
    print("ROUND NUMBER {}:".format(round_count))
    return print('\n'.join(map(str,game2)))

format_round(start_pos)

ROUND NUMBER 1:
.  #  .  #  #  #  .  .  #  .  #  #  #  #  .  .  #  #  .  .
.  .  #  .  .  .  .  #  #  .  .  #  #  .  #  #  #  .  .  #
.  #  .  #  .  .  .  .  .  .  #  .  .  #  #  .  #  #  #  .
.  .  .  #  #  #  .  .  .  .  #  #  .  #  #  .  .  .  #  #
#  #  #  #  #  #  .  #  .  #  #  #  #  #  .  .  .  #  #  #
.  #  #  #  .  .  .  #  .  #  #  .  .  #  #  #  .  .  .  .
.  .  .  .  .  .  .  #  .  .  #  .  .  #  #  #  .  .  #  .
.  .  .  #  #  #  .  .  .  #  #  #  .  .  .  .  .  #  .  .
.  .  .  #  .  .  #  #  .  .  #  .  .  #  .  #  #  .  #  #
#  .  #  #  #  .  .  .  .  .  #  #  #  .  #  #  .  #  .  #
.  #  #  .  .  #  #  .  #  .  .  #  #  #  .  #  .  .  #  #
.  .  #  #  #  #  #  .  #  #  #  #  .  #  .  .  #  .  .  #
#  #  #  #  .  #  #  #  #  #  .  #  .  #  .  .  #  #  .  #
.  #  .  #  #  #  #  .  .  #  #  #  .  .  .  .  #  .  .  #
#  #  #  #  .  .  .  .  .  .  #  .  #  #  #  #  .  #  .  .
#  #  #  #  .  #  .  .  #  #  .  #  .  #  #  .  #  #  .  .
#  .  .  #  #  .  .  #  .  #  #  #  #  #

In [33]:
game = start_pos

def check_position(line, pos): # e.g. 2, 4
    counter = 0
    try:
        counter += numpy.sum(game[line-1][pos-1:pos+2])
    except:
        pass
    try:
        counter += numpy.sum(game[line+1][pos-1:pos+2])
    except:
        pass
    counter += numpy.sum(game[line][pos-1:pos+2])
    
    print(fields_to_check)
    fields_to_check = sum(reduce(lambda x,y: x+y,fields_to_check))
    fields_to_check = fields_to_check - int(game[line][pos])
    print(fields_to_check)
    return fields_to_check

In [22]:
def alive_or_dead(line, pos):
    life_around = check_position(line, pos)
    if life_around == 3:
        return 1
    if life_around == 2:
        return game[line][pos]
    if life_around <= 1 or life_around >= 4:
        return 0

In [34]:
print(check_position(width, width))

IndexError: list index out of range

In [ ]:
this_round = start_pos

def calculate_next_round(current_round):
    next_round = copy.deepcopy(this_round)
    for line in range(width):
        for pos in range(height):
            next_round[line][pos] = alive_or_dead(int(line), int(pos))
    return next_round


next_round = calculate_next_round(this_round)
round_count += 1

format_round(next_round)